In [1]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    LOC,
    NamesExtractor,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,

    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)
dates_extractor = DatesExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
df = pd.read_csv('/Users/Gixx/Desktop/news.csv', sep=';', encoding='utf8')

In [5]:
df.head(10)

title  \
0  Информация по искусственному осеменению коров ...   
1  ОМВД России по Чаинскому району   приглашает ж...   
2                         С Наступающим Новым годом!   
3              ВСЕЙ СЕМЬЕЙ — ВСЕГДА ДРУЖНО И ВЕСЕЛО!   
4  Открыта переправа через речку Чая в селе Гришкино   
5                          Будьте осторожны с огнём!   
6                             С Днём Конституции РФ!   
7     Открыта ледовая переправа Подгорное- Черёмушки   
8  Военная служба по контракту – работа настоящих...   
9                            С Днём Героев Отечества   

                                                link             date  \
0  https://www.chainsk.tom.ru/news/admrna/informa...  29 декабря 2023   
1  https://www.chainsk.tom.ru/news/admrna/omvd_ro...  29 декабря 2023   
2  https://www.chainsk.tom.ru/news/admrna/s_nastu...  29 декабря 2023   
3  https://www.chainsk.tom.ru/news/admrna/vsej_se...  18 декабря 2023   
4  https://www.chainsk.tom.ru/news/admrna/otkryta...  18 декабря 2023   
5  https://www.chainsk.tom.ru/news/admrna/budte_o...  13 декабря 2023   
6  https://www.chainsk.tom.ru/news/admrna/s_dnem_...  12 декабря 2023   
7  https://www.chainsk.tom.ru/news/admrna/otkryta...  12 декабря 2023   
8  https://www.chainsk.tom.ru/news/admrna/voennaj...  11 декабря 2023   
9  https://www.chainsk.tom.ru/news/admrna/s_dnjom...  09 декабря 2023   

                              author_link                          author  
0  https://www.chainsk.tom.ru/news/admrna  Администрация Чаинского района  
1  https://www.chainsk.tom.ru/news/admrna  Администрация Чаинского района  
2  https://www.chainsk.tom.ru/news/admrna  Администрация Чаинского района  
3  https://www.chainsk.tom.ru/news/admrna  Администрация Чаинского района  
4  https://www.chainsk.tom.ru/news/admrna  Администрация Чаинского района  
5  https://www.chainsk.tom.ru/news/admrna  Администрация Чаинского района  
6  https://www.chainsk.tom.ru/news/admrna  Администрация Чаинского района  
7  https://www.chainsk.tom.ru/news/admrna  Администрация Чаинского района  
8  https://www.chainsk.tom.ru/news/admrna  Администрация Чаинского района  
9  https://www.chainsk.tom.ru/news/admrna  Администрация Чаинского района

In [21]:
df.to_string(buf=None, columns=None, col_space=None, header=True, index=True, na_rep='NaN', formatters=None, float_format=None, sparsify=None, index_names=True, justify=None, max_rows=None, max_cols=None, show_dimensions=False, decimal='.', line_width=None, min_rows=None, max_colwidth=None, encoding=None)

'                                                                                                                                                      title                                                                                                                                                                                           link              date                             author_link                          author\n0                                                                                                 Информация по искусственному осеменению коров в 2024 году                                                                                             https://www.chainsk.tom.ru/news/admrna/informacija_po_iskusstvennomu_osemeneniju_korov_v_2024_godu   29 декабря 2023  https://www.chainsk.tom.ru/news/admrna  Администрация Чаинского района\n1                                                   ОМВД России по Чаинскому району\xa0  приглашает\xa0жителей\xa0района\xa

In [57]:
#Здесь я не смог догадаться как взять и обработать все заголовки, если я просто ставлю title, то он требует от меня строчку, а не серию объектов, ставлю от 0 до 800, он выдает ошибку мультииндекса
text = df.title[0]
text

'Информация по искусственному осеменению коров в 2024 году'

In [43]:
df["title"]=df["title"].values.astype('str')

In [44]:
df.dtypes

title          object
link           object
date           object
author_link    object
author         object
dtype: object

In [48]:
#Передаем текст библиотеке
doc = Doc(text)
doc.segment(segmenter) 
doc.tag_morph(morph_tagger)
doc.parse_syntax(syntax_parser)
doc.tag_ner(ner_tagger)

In [58]:
#Разбиваем текст на токены, показываем первые 5
doc.segment(segmenter)
display(doc.tokens[:5])

[DocToken(stop=10, text='Информация'),
 DocToken(start=11, stop=13, text='по'),
 DocToken(start=14, stop=28, text='искусственному'),
 DocToken(start=29, stop=39, text='осеменению'),
 DocToken(start=40, stop=45, text='коров')]

In [59]:
#Разбиваем на предложения
display(doc.sents[:5])

[DocSent(stop=57, text='Информация по искусственному осеменению коров в 2..., tokens=[...])]

In [60]:
#Морфологический разбор слов
doc.tag_morph(morph_tagger)
display(doc.tokens[:5])

[DocToken(stop=10, text='Информация', pos='NOUN', feats=<Inan,Nom,Fem,Sing>),
 DocToken(start=11, stop=13, text='по', pos='ADP'),
 DocToken(start=14, stop=28, text='искусственному', pos='ADJ', feats=<Dat,Pos,Masc,Sing>),
 DocToken(start=29, stop=39, text='осеменению', pos='NOUN', feats=<Inan,Dat,Neut,Sing>),
 DocToken(start=40, stop=45, text='коров', pos='NOUN', feats=<Anim,Gen,Fem,Plur>)]

In [63]:
#Лемматизация
for token in doc.tokens:
    token.lemmatize(morph_vocab)
    
{_.text: _.lemma for _ in doc.tokens}

{'Информация': 'информация',
 'по': 'по',
 'искусственному': 'искусственный',
 'осеменению': 'осеменение',
 'коров': 'корова',
 'в': 'в',
 '2024': '2024',
 'году': 'год'}

In [64]:
#Пробуем извлечь даты с помощью библиотеки, можно извлекать интересующие объекты, например имена. В моем примере просто мало, что можно извлечь
matches = dates_extractor(text)
facts = [i.fact.as_json for i in matches]
facts

[OrderedDict([('year', 2024)])]